In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import sklearn.svm as svm
import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score, cross_validate


In [ ]:
data = pd.read_csv('/content/drive/My Drive/미래에셋/df_cat2.csv')
data

,Unnamed: 0,X,ID,nur_hosp_yn,ac_ctr_diff,hsp_avg_optt_bilg_isamt_s,hsp_avg_surop_bilg_isamt_s,ar_rclss_cd,fds_cust_yn,hspz_dys_s,inamt_nvcd,hsp_avg_diag_bilg_isamt_s,blrs_cd,dsas_ltwt_gcd,dsas_avg_diag_bilg_isamt_s,dsas_acd_rst_dcd,base_ym,kcd_gcd,hsp_avg_hspz_bilg_isamt_s,optt_blcnt_s,mtad_cntr_yn,heltp_pf_ntyn,prm_nvcd,surop_blcnt_s,mdct_inu_rclss_dcd,dsas_avg_optt_bilg_isamt_s,isrd_age_dcd,hspz_blcnt_s,dsas_avg_surop_bilg_isamt_s,urlb_fc_yn,dsas_avg_hspz_bilg_isamt_s,smrtg_5y_passed_yn,ac_rst_diff,bilg_isamt_s,optt_nbtm_s,target,dsas_acd_rst_dcd2,prm_nvcd2,inamt_nvcd2,kcd_gcd_2,blrs_cd_2
0,1,1,0,0,6,0.122483,0.000000,1,0,0.296543,4,0.000000,12,1,0.682743,1,201901,3,0.956319,1.098612,0,0,1,0.000000,2,0.788548,3,2.397895,0.774773,0,0.815896,0,0,0.353470,0.619016,1,2,1,4,1,1
1,2,2,1,1,6,0.000000,0.455524,4,0,0.000000,4,0.000000,2,3,0.000000,22,201901,8,0.000000,0.000000,0,0,1,0.693147,1,0.000000,4,0.000000,0.140457,0,0.002297,0,0,0.620684,0.000000,0,5,1,4,3,3
2,3,3,2,1,6,0.000000,0.253479,1,1,0.000000,6,0.000000,2,3,0.000000,22,201901,8,0.000000,0.000000,0,0,1,0.693147,1,0.000000,4,0.000000,0.140457,0,0.002297,0,0,0.357744,0.000000,1,5,1,5,3,3
3,4,4,3,0,4,0.019508,0.065788,2,0,0.590893,6,0.057042,8,3,0.000000,23,201901,8,0.447119,0.000000,0,0,1,0.000000,2,0.000000,9,0.893800,0.082777,0,0.175633,0,0,0.223863,0.000000,1,4,1,5,3,5
4,5,5,4,0,5,0.000000,0.000000,9,0,0.348330,2,0.000000,8,3,0.000000,23,201901,8,0.369078,0.000000,0,0,2,0.000000,2,0.000000,0,0.847284,0.082777,0,0.175633,0,0,0.031789,0.000000,1,4,2,2,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377923,377924,377924,377923,0,6,0.587620,0.722123,1,0,0.135754,2,0.802898,11,1,0.746356,1,201911,3,0.380216,0.000000,0,0,1,0.693147,2,0.770571,3,0.843376,0.830951,0,0.772882,0,0,2.356296,0.000000,2,2,1,2,1,1
377924,377925,377925,377924,0,0,1.195346,0.832605,9,0,1.009198,99,0.443724,9,1,0.730491,1,201904,3,0.579082,0.000000,0,0,99,0.581992,2,0.798768,7,2.036886,0.779829,0,0.703048,0,6,0.000000,0.000000,2,2,99,99,1,1
377925,377926,377926,377925,1,0,0.000000,0.000000,9,0,3.164909,99,0.000000,8,1,0.610689,6,201907,9,2.457518,0.000000,0,0,99,0.000000,2,0.000000,7,0.544299,0.704137,0,0.939812,0,0,0.000000,0.000000,2,3,99,99,3,5
377926,377927,377927,377926,0,6,0.866428,0.879461,4,0,0.079920,1,0.631910,10,1,0.743650,1,201909,3,0.525202,0.000000,0,0,1,0.693147,3,0.810886,3,0.510846,0.822683,0,0.790274,0,0,1.314772,0.000000,2,2,1,1,1,5


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377928 entries, 0 to 377927
Data columns (total 41 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   377928 non-null  int64  
 1   X                            377928 non-null  int64  
 2   ID                           377928 non-null  int64  
 3   nur_hosp_yn                  377928 non-null  int64  
 4   ac_ctr_diff                  377928 non-null  int64  
 5   hsp_avg_optt_bilg_isamt_s    377928 non-null  float64
 6   hsp_avg_surop_bilg_isamt_s   377928 non-null  float64
 7   ar_rclss_cd                  377928 non-null  int64  
 8   fds_cust_yn                  377928 non-null  int64  
 9   hspz_dys_s                   377928 non-null  float64
 10  inamt_nvcd                   377928 non-null  int64  
 11  hsp_avg_diag_bilg_isamt_s    377928 non-null  float64
 12  blrs_cd                      377928 non-null  int64  
 13 

In [ ]:
# 인덱스 변수들, 범주화 전 변수들 제거

data.drop(['Unnamed: 0'],axis='columns',inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377928 entries, 0 to 377927
Data columns (total 40 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   X                            377928 non-null  int64  
 1   ID                           377928 non-null  int64  
 2   nur_hosp_yn                  377928 non-null  int64  
 3   ac_ctr_diff                  377928 non-null  int64  
 4   hsp_avg_optt_bilg_isamt_s    377928 non-null  float64
 5   hsp_avg_surop_bilg_isamt_s   377928 non-null  float64
 6   ar_rclss_cd                  377928 non-null  int64  
 7   fds_cust_yn                  377928 non-null  int64  
 8   hspz_dys_s                   377928 non-null  float64
 9   inamt_nvcd                   377928 non-null  int64  
 10  hsp_avg_diag_bilg_isamt_s    377928 non-null  float64
 11  blrs_cd                      377928 non-null  int64  
 12  dsas_ltwt_gcd                377928 non-null  int64  
 13 

In [ ]:
# factor형 변수 변환, 날짜형 변수 변환
cat_f = ['dsas_ltwt_gcd','ar_rclss_cd','mdct_inu_rclss_dcd','nur_hosp_yn', 'isrd_age_dcd',
          'fds_cust_yn','smrtg_5y_passed_yn','mtad_cntr_yn','heltp_pf_ntyn',
          'ac_ctr_diff','ac_rst_diff','urlb_fc_yn', 'target','kcd_gcd_2','blrs_cd_2',
          'dsas_acd_rst_dcd2','prm_nvcd2','inamt_nvcd2']

for colnm in cat_f:
  data[colnm]= data[colnm].astype('category')


data['base_ym']=pd.to_datetime(data['base_ym'])

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377928 entries, 0 to 377927
Data columns (total 40 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   X                            377928 non-null  int64   
 1   ID                           377928 non-null  int64   
 2   nur_hosp_yn                  377928 non-null  category
 3   ac_ctr_diff                  377928 non-null  category
 4   hsp_avg_optt_bilg_isamt_s    377928 non-null  float64 
 5   hsp_avg_surop_bilg_isamt_s   377928 non-null  float64 
 6   ar_rclss_cd                  377928 non-null  category
 7   fds_cust_yn                  377928 non-null  category
 8   hspz_dys_s                   377928 non-null  float64 
 9   inamt_nvcd                   377928 non-null  int64   
 10  hsp_avg_diag_bilg_isamt_s    377928 non-null  float64 
 11  blrs_cd                      377928 non-null  int64   
 12  dsas_ltwt_gcd                377928 non-null

In [ ]:
data.drop(['dsas_acd_rst_dcd','prm_nvcd','inamt_nvcd','kcd_gcd','blrs_cd'],axis=1,
          inplace=True)
data.info()

In [ ]:
data.drop(['X','ID'],axis=1,inplace=True)
data.info()
## 0,1 불균형한 변수들도 존재함

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377928 entries, 0 to 377927
Data columns (total 33 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   nur_hosp_yn                  377928 non-null  category      
 1   ac_ctr_diff                  377928 non-null  category      
 2   hsp_avg_optt_bilg_isamt_s    377928 non-null  float64       
 3   hsp_avg_surop_bilg_isamt_s   377928 non-null  float64       
 4   ar_rclss_cd                  377928 non-null  category      
 5   fds_cust_yn                  377928 non-null  category      
 6   hspz_dys_s                   377928 non-null  float64       
 7   hsp_avg_diag_bilg_isamt_s    377928 non-null  float64       
 8   dsas_ltwt_gcd                377928 non-null  category      
 9   dsas_avg_diag_bilg_isamt_s   377928 non-null  float64       
 10  base_ym                      377928 non-null  datetime64[ns]
 11  hsp_avg_hspz_bilg_isamt_s 

#### 변수중요도 낮은애들 제거

In [ ]:
# 범주형 0,1 범주 불균형한 변수들 제거
data.drop(['fds_cust_yn','smrtg_5y_passed_yn','mtad_cntr_yn','heltp_pf_ntyn','urlb_fc_yn'],axis=1,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377928 entries, 0 to 377927
Data columns (total 28 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   nur_hosp_yn                  377928 non-null  category      
 1   ac_ctr_diff                  377928 non-null  category      
 2   hsp_avg_optt_bilg_isamt_s    377928 non-null  float64       
 3   hsp_avg_surop_bilg_isamt_s   377928 non-null  float64       
 4   ar_rclss_cd                  377928 non-null  category      
 5   hspz_dys_s                   377928 non-null  float64       
 6   hsp_avg_diag_bilg_isamt_s    377928 non-null  float64       
 7   dsas_ltwt_gcd                377928 non-null  category      
 8   dsas_avg_diag_bilg_isamt_s   377928 non-null  float64       
 9   base_ym                      377928 non-null  datetime64[ns]
 10  hsp_avg_hspz_bilg_isamt_s    377928 non-null  float64       
 11  optt_blcnt_s              

In [ ]:
# 상관관계 높은 변수 제거
#data.drop(['dsas_avg_diag_bilg_isamt_s','dsas_avg_hspz_bilg_isamt_s'],axis=1,inplace=True)
#data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377928 entries, 0 to 377927
Data columns (total 26 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   nur_hosp_yn                  377928 non-null  category      
 1   ac_ctr_diff                  377928 non-null  category      
 2   hsp_avg_optt_bilg_isamt_s    377928 non-null  float64       
 3   hsp_avg_surop_bilg_isamt_s   377928 non-null  float64       
 4   ar_rclss_cd                  377928 non-null  category      
 5   hspz_dys_s                   377928 non-null  float64       
 6   hsp_avg_diag_bilg_isamt_s    377928 non-null  float64       
 7   dsas_ltwt_gcd                377928 non-null  category      
 8   base_ym                      377928 non-null  datetime64[ns]
 9   hsp_avg_hspz_bilg_isamt_s    377928 non-null  float64       
 10  optt_blcnt_s                 377928 non-null  float64       
 11  surop_blcnt_s             

###  catboost

In [ ]:
pip install catboost

     |████████████████████████████████| 66.1MB 70kB/s 


In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

In [ ]:
# set x and y for CatBoost
X = data.drop(['target'],axis=1)
y = data['target']

In [ ]:
# scaling - z score
num_f =['bilg_isamt_s','hspz_dys_s','hsp_avg_hspz_bilg_isamt_s',
          'hsp_avg_optt_bilg_isamt_s','hsp_avg_surop_bilg_isamt_s','hsp_avg_diag_bilg_isamt_s',
          'dsas_avg_optt_bilg_isamt_s','dsas_avg_surop_bilg_isamt_s','dsas_avg_diag_bilg_isamt_s','dsas_avg_hspz_bilg_isamt_s',
          'hspz_blcnt_s','surop_blcnt_s']

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X[num_f] = scaler.fit_transform(X[num_f])

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377928 entries, 0 to 377927
Data columns (total 25 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   nur_hosp_yn                  377928 non-null  category      
 1   ac_ctr_diff                  377928 non-null  category      
 2   hsp_avg_optt_bilg_isamt_s    377928 non-null  float64       
 3   hsp_avg_surop_bilg_isamt_s   377928 non-null  float64       
 4   ar_rclss_cd                  377928 non-null  category      
 5   hspz_dys_s                   377928 non-null  float64       
 6   hsp_avg_diag_bilg_isamt_s    377928 non-null  float64       
 7   dsas_ltwt_gcd                377928 non-null  category      
 8   base_ym                      377928 non-null  datetime64[ns]
 9   hsp_avg_hspz_bilg_isamt_s    377928 non-null  float64       
 10  optt_blcnt_s                 377928 non-null  float64       
 11  surop_blcnt_s             

In [ ]:
# split dataset
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size = 0.2, random_state=1234)


In [ ]:
import time
# train CatBoost
start = time.time()
cat_features = np.where(X.dtypes.astype("str").isin(["category","object"]))[0]
CatBoost = CatBoostClassifier(iterations=994,learning_rate=0.01,grow_policy = 'Depthwise',depth=8,
                           loss_function='MultiClass',eval_metric='TotalF1',random_seed=1234)
#model.fit(train_data,
 #         train_labels,
 #         eval_set=eval_dataset,
 #         verbose=False)

model=CatBoost.fit(X = X_train, y = y_train, cat_features = cat_features, eval_set=(X_valid, y_valid))

CatBoost_Runtime = time.time() - start
#print(CatBoost.get_best_iteration())

0:	learn: 0.7336007	test: 0.7317945	best: 0.7317945 (0)	total: 4.02s	remaining: 1h 6m 31s
1:	learn: 0.7403373	test: 0.7385042	best: 0.7385042 (1)	total: 7.57s	remaining: 1h 2m 35s
2:	learn: 0.7425135	test: 0.7410629	best: 0.7410629 (2)	total: 11.1s	remaining: 1h 1m 16s
3:	learn: 0.7459794	test: 0.7448257	best: 0.7448257 (3)	total: 14.9s	remaining: 1h 1m 28s
4:	learn: 0.7466666	test: 0.7449345	best: 0.7449345 (4)	total: 18.5s	remaining: 1h 1m 7s
5:	learn: 0.7486238	test: 0.7464407	best: 0.7464407 (5)	total: 22.1s	remaining: 1h 39s
6:	learn: 0.7505458	test: 0.7482663	best: 0.7482663 (6)	total: 25.7s	remaining: 1h 28s
7:	learn: 0.7518044	test: 0.7494156	best: 0.7494156 (7)	total: 29.3s	remaining: 1h 8s
8:	learn: 0.7534380	test: 0.7508805	best: 0.7508805 (8)	total: 33.1s	remaining: 1h 21s
9:	learn: 0.7552444	test: 0.7526326	best: 0.7526326 (9)	total: 37s	remaining: 1h 36s
10:	learn: 0.7563269	test: 0.7535167	best: 0.7535167 (10)	total: 40.8s	remaining: 1h 43s
11:	learn: 0.7576729	test: 0.7

###### test data불러오기

In [ ]:
# test 데이터 불러오기
testData=pd.read_csv('/content/drive/My Drive/미래에셋/test_cat2.csv')
testData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22072 entries, 0 to 22071
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   22072 non-null  int64  
 1   ID                           22072 non-null  int64  
 2   nur_hosp_yn                  22072 non-null  int64  
 3   ac_ctr_diff                  22072 non-null  int64  
 4   hsp_avg_optt_bilg_isamt_s    22072 non-null  float64
 5   hsp_avg_surop_bilg_isamt_s   22072 non-null  float64
 6   ar_rclss_cd                  22072 non-null  int64  
 7   fds_cust_yn                  22072 non-null  int64  
 8   hspz_dys_s                   22072 non-null  float64
 9   inamt_nvcd                   22072 non-null  int64  
 10  hsp_avg_diag_bilg_isamt_s    22072 non-null  float64
 11  blrs_cd                      22072 non-null  int64  
 12  dsas_ltwt_gcd                22072 non-null  int64  
 13  dsas_avg_diag_bi

In [ ]:
# factor형 변수 변환
cat_f = ['dsas_ltwt_gcd','ar_rclss_cd','mdct_inu_rclss_dcd','nur_hosp_yn', 'isrd_age_dcd',
          'fds_cust_yn','smrtg_5y_passed_yn','mtad_cntr_yn','heltp_pf_ntyn',
          'ac_ctr_diff','ac_rst_diff','urlb_fc_yn','kcd_gcd_2','blrs_cd_2',
          'dsas_acd_rst_dcd2','prm_nvcd2','inamt_nvcd2']

for colnm in cat_f:
  testData[colnm]= testData[colnm].astype('category')

testData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22072 entries, 0 to 22071
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   Unnamed: 0                   22072 non-null  int64   
 1   ID                           22072 non-null  int64   
 2   nur_hosp_yn                  22072 non-null  category
 3   ac_ctr_diff                  22072 non-null  category
 4   hsp_avg_optt_bilg_isamt_s    22072 non-null  float64 
 5   hsp_avg_surop_bilg_isamt_s   22072 non-null  float64 
 6   ar_rclss_cd                  22072 non-null  category
 7   fds_cust_yn                  22072 non-null  category
 8   hspz_dys_s                   22072 non-null  float64 
 9   inamt_nvcd                   22072 non-null  int64   
 10  hsp_avg_diag_bilg_isamt_s    22072 non-null  float64 
 11  blrs_cd                      22072 non-null  int64   
 12  dsas_ltwt_gcd                22072 non-null  category
 13  d

In [ ]:
# 불필요한 변수들, 재범주화 전 변수들 제거
testData.drop(['Unnamed: 0','ID','inamt_nvcd','blrs_cd','dsas_acd_rst_dcd','kcd_gcd','prm_nvcd'],
              axis=1, inplace=True)
testData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22072 entries, 0 to 22071
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   nur_hosp_yn                  22072 non-null  category
 1   ac_ctr_diff                  22072 non-null  category
 2   hsp_avg_optt_bilg_isamt_s    22072 non-null  float64 
 3   hsp_avg_surop_bilg_isamt_s   22072 non-null  float64 
 4   ar_rclss_cd                  22072 non-null  category
 5   fds_cust_yn                  22072 non-null  category
 6   hspz_dys_s                   22072 non-null  float64 
 7   hsp_avg_diag_bilg_isamt_s    22072 non-null  float64 
 8   dsas_ltwt_gcd                22072 non-null  category
 9   dsas_avg_diag_bilg_isamt_s   22072 non-null  float64 
 10  base_ym                      22072 non-null  int64   
 11  hsp_avg_hspz_bilg_isamt_s    22072 non-null  float64 
 12  optt_blcnt_s                 22072 non-null  float64 
 13  m

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 302342 entries, 223382 to 165158
Data columns (total 23 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   nur_hosp_yn                  302342 non-null  category      
 1   ac_ctr_diff                  302342 non-null  category      
 2   hsp_avg_optt_bilg_isamt_s    302342 non-null  float64       
 3   hsp_avg_surop_bilg_isamt_s   302342 non-null  float64       
 4   ar_rclss_cd                  302342 non-null  category      
 5   hspz_dys_s                   302342 non-null  float64       
 6   hsp_avg_diag_bilg_isamt_s    302342 non-null  float64       
 7   base_ym                      302342 non-null  datetime64[ns]
 8   hsp_avg_hspz_bilg_isamt_s    302342 non-null  float64       
 9   optt_blcnt_s                 302342 non-null  float64       
 10  surop_blcnt_s                302342 non-null  float64       
 11  mdct_inu_rclss_dcd   

In [ ]:
testData['base_ym']=pd.to_datetime(testData['base_ym'])
testData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22072 entries, 0 to 22071
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   nur_hosp_yn                  22072 non-null  category      
 1   ac_ctr_diff                  22072 non-null  category      
 2   hsp_avg_optt_bilg_isamt_s    22072 non-null  float64       
 3   hsp_avg_surop_bilg_isamt_s   22072 non-null  float64       
 4   ar_rclss_cd                  22072 non-null  category      
 5   fds_cust_yn                  22072 non-null  category      
 6   hspz_dys_s                   22072 non-null  float64       
 7   hsp_avg_diag_bilg_isamt_s    22072 non-null  float64       
 8   dsas_ltwt_gcd                22072 non-null  category      
 9   dsas_avg_diag_bilg_isamt_s   22072 non-null  float64       
 10  base_ym                      22072 non-null  datetime64[ns]
 11  hsp_avg_hspz_bilg_isamt_s    22072 non-nu

In [ ]:
# 범주형 0,1 범주 불균형한 변수들 제거
testData.drop(['fds_cust_yn','smrtg_5y_passed_yn','mtad_cntr_yn','heltp_pf_ntyn','urlb_fc_yn'],axis=1,inplace=True)
testData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22072 entries, 0 to 22071
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   nur_hosp_yn                  22072 non-null  category      
 1   ac_ctr_diff                  22072 non-null  category      
 2   hsp_avg_optt_bilg_isamt_s    22072 non-null  float64       
 3   hsp_avg_surop_bilg_isamt_s   22072 non-null  float64       
 4   ar_rclss_cd                  22072 non-null  category      
 5   hspz_dys_s                   22072 non-null  float64       
 6   hsp_avg_diag_bilg_isamt_s    22072 non-null  float64       
 7   dsas_ltwt_gcd                22072 non-null  category      
 8   dsas_avg_diag_bilg_isamt_s   22072 non-null  float64       
 9   base_ym                      22072 non-null  datetime64[ns]
 10  hsp_avg_hspz_bilg_isamt_s    22072 non-null  float64       
 11  optt_blcnt_s                 22072 non-nu

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 302342 entries, 223382 to 165158
Data columns (total 23 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   nur_hosp_yn                  302342 non-null  category      
 1   ac_ctr_diff                  302342 non-null  category      
 2   hsp_avg_optt_bilg_isamt_s    302342 non-null  float64       
 3   hsp_avg_surop_bilg_isamt_s   302342 non-null  float64       
 4   ar_rclss_cd                  302342 non-null  category      
 5   hspz_dys_s                   302342 non-null  float64       
 6   hsp_avg_diag_bilg_isamt_s    302342 non-null  float64       
 7   base_ym                      302342 non-null  datetime64[ns]
 8   hsp_avg_hspz_bilg_isamt_s    302342 non-null  float64       
 9   optt_blcnt_s                 302342 non-null  float64       
 10  surop_blcnt_s                302342 non-null  float64       
 11  mdct_inu_rclss_dcd   

In [ ]:
print(X_train.columns)
print(testData.columns)

Index(['nur_hosp_yn', 'ac_ctr_diff', 'hsp_avg_optt_bilg_isamt_s',
       'hsp_avg_surop_bilg_isamt_s', 'ar_rclss_cd', 'fds_cust_yn',
       'hspz_dys_s', 'hsp_avg_diag_bilg_isamt_s', 'dsas_ltwt_gcd',
       'dsas_avg_diag_bilg_isamt_s', 'base_ym', 'hsp_avg_hspz_bilg_isamt_s',
       'optt_blcnt_s', 'mtad_cntr_yn', 'heltp_pf_ntyn', 'surop_blcnt_s',
       'mdct_inu_rclss_dcd', 'dsas_avg_optt_bilg_isamt_s', 'isrd_age_dcd',
       'hspz_blcnt_s', 'dsas_avg_surop_bilg_isamt_s', 'urlb_fc_yn',
       'dsas_avg_hspz_bilg_isamt_s', 'smrtg_5y_passed_yn', 'ac_rst_diff',
       'bilg_isamt_s', 'optt_nbtm_s', 'dsas_acd_rst_dcd2', 'prm_nvcd2',
       'inamt_nvcd2', 'kcd_gcd_2', 'blrs_cd_2'],
      dtype='object')
Index(['nur_hosp_yn', 'ac_ctr_diff', 'hsp_avg_optt_bilg_isamt_s',
       'hsp_avg_surop_bilg_isamt_s', 'ar_rclss_cd', 'fds_cust_yn',
       'hspz_dys_s', 'hsp_avg_diag_bilg_isamt_s', 'dsas_ltwt_gcd',
       'dsas_avg_diag_bilg_isamt_s', 'base_ym', 'hsp_avg_hspz_bilg_isamt_s',
       'optt_

In [ ]:
testData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22072 entries, 0 to 22071
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   nur_hosp_yn                  22072 non-null  category      
 1   ac_ctr_diff                  22072 non-null  category      
 2   hsp_avg_optt_bilg_isamt_s    22072 non-null  float64       
 3   hsp_avg_surop_bilg_isamt_s   22072 non-null  float64       
 4   ar_rclss_cd                  22072 non-null  category      
 5   fds_cust_yn                  22072 non-null  category      
 6   hspz_dys_s                   22072 non-null  float64       
 7   hsp_avg_diag_bilg_isamt_s    22072 non-null  float64       
 8   dsas_ltwt_gcd                22072 non-null  category      
 9   dsas_avg_diag_bilg_isamt_s   22072 non-null  float64       
 10  base_ym                      22072 non-null  datetime64[ns]
 11  hsp_avg_hspz_bilg_isamt_s    22072 non-nu

In [ ]:
# scaling
testData[num_f] = scaler.transform(testData[num_f])

In [ ]:
testData[num_f]

,bilg_isamt_s,hspz_dys_s,hsp_avg_hspz_bilg_isamt_s,hsp_avg_optt_bilg_isamt_s,hsp_avg_surop_bilg_isamt_s,hsp_avg_diag_bilg_isamt_s,dsas_avg_hspz_bilg_isamt_s,dsas_avg_optt_bilg_isamt_s,dsas_avg_surop_bilg_isamt_s,dsas_avg_diag_bilg_isamt_s,hspz_blcnt_s,surop_blcnt_s
0,-0.309862,-0.734349,-1.189656,-0.669998,-0.587309,-0.818911,-0.943573,-0.693320,-0.821773,-0.717600,-1.175882,0.595193
1,0.645266,-0.734349,-0.081464,1.238052,1.866155,1.761741,0.052013,0.573135,1.199402,1.502841,-1.175882,0.530230
2,4.619224,-0.734349,0.377072,1.301828,1.411843,1.307026,1.176865,1.635013,1.510672,1.498619,-1.175882,-1.135574
3,1.547062,-0.429894,-0.703970,-0.431561,0.452557,0.152419,1.176865,1.635013,1.510672,1.498619,0.104860,0.830321
4,-0.290921,-0.734349,-1.189656,-0.669998,-0.694601,-0.818911,-0.943573,-0.693320,-0.821773,-0.717600,-1.175882,0.530230
...,...,...,...,...,...,...,...,...,...,...,...,...
22067,-0.416193,-0.734349,-1.137567,-0.669998,-0.798735,-0.818911,-0.943573,-0.693320,-0.821773,-0.717600,-1.175882,0.530230
22068,-0.817896,-0.734349,0.089359,-0.669998,-0.038457,-0.818911,-0.943573,-0.693320,-0.821773,-0.717600,0.104860,-1.135574
22069,-0.536333,-0.734349,-1.189656,-0.669998,-1.096007,-0.818911,-1.321543,-0.693320,-0.802852,-0.717600,-1.175882,1.504663
22070,1.673286,-0.734349,0.494505,1.394650,1.569676,1.015346,0.701940,1.198311,0.956521,1.040664,-1.175882,0.101469


In [ ]:
test_pred=CatBoost.predict(testData)
test_pred

array([[0],
       [1],
       [1],
       ...,
       [1],
       [2],
       [1]])

In [ ]:
test_pred2=np.zeros(len(test_pred))
test_pred2

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
for i in range(len(test_pred)):
  test_pred2[i]= list(test_pred[i])[0]

In [ ]:
test_pred2

array([0., 1., 1., ..., 1., 2., 1.])

In [ ]:
cat_res=pd.DataFrame({'target':test_pred2})
cat_res['target']=cat_res['target'].astype('int')
cat_res['target']=cat_res['target'].astype('category')
cat_res

,target
0,0.0
1,1.0
2,1.0
3,1.0
4,0.0
...,...
22067,0.0
22068,1.0
22069,1.0
22070,2.0


In [ ]:
cat_res.to_csv('catboost_cat2_parameter수정_biased변수제거.csv')

### 질병구분코드 구분해서 개별모델 돌리기

In [ ]:
PATH = '/content/drive/My Drive/미래에셋/'

acd1=pd.read_csv(PATH+'acd1_cat2.csv')  ## 질병구분코드 2이면 acd2_cat2.csv 로 경로만 수정
acd1.info()

In [ ]:
acd1.drop(['X','V1','ID','Unnamed: 0'],axis=1, inplace=True)
acd1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1759 entries, 0 to 1758
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   nur_hosp_yn                  1759 non-null   int64  
 1   ac_ctr_diff                  1759 non-null   int64  
 2   hsp_avg_optt_bilg_isamt_s    1759 non-null   float64
 3   hsp_avg_surop_bilg_isamt_s   1759 non-null   float64
 4   ar_rclss_cd                  1759 non-null   int64  
 5   fds_cust_yn                  1759 non-null   int64  
 6   hspz_dys_s                   1759 non-null   float64
 7   inamt_nvcd                   1759 non-null   int64  
 8   hsp_avg_diag_bilg_isamt_s    1759 non-null   float64
 9   blrs_cd                      1759 non-null   int64  
 10  dsas_ltwt_gcd                1759 non-null   int64  
 11  dsas_avg_diag_bilg_isamt_s   1759 non-null   float64
 12  dsas_acd_rst_dcd             1759 non-null   int64  
 13  base_ym           

In [ ]:
# factor형 변수 변환
cat_f = ['dsas_ltwt_gcd','ar_rclss_cd','mdct_inu_rclss_dcd','nur_hosp_yn', 'isrd_age_dcd',
          'fds_cust_yn','smrtg_5y_passed_yn','mtad_cntr_yn','heltp_pf_ntyn',
          'ac_ctr_diff','ac_rst_diff','urlb_fc_yn', 'target','kcd_gcd_2','blrs_cd_2',
          'dsas_acd_rst_dcd2','prm_nvcd2','inamt_nvcd2']

for colnm in cat_f:
  acd1[colnm]= acd1[colnm].astype('category')

acd1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1759 entries, 0 to 1758
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   nur_hosp_yn                  1759 non-null   category
 1   ac_ctr_diff                  1759 non-null   category
 2   hsp_avg_optt_bilg_isamt_s    1759 non-null   float64 
 3   hsp_avg_surop_bilg_isamt_s   1759 non-null   float64 
 4   ar_rclss_cd                  1759 non-null   category
 5   fds_cust_yn                  1759 non-null   category
 6   hspz_dys_s                   1759 non-null   float64 
 7   inamt_nvcd                   1759 non-null   int64   
 8   hsp_avg_diag_bilg_isamt_s    1759 non-null   float64 
 9   blrs_cd                      1759 non-null   int64   
 10  dsas_ltwt_gcd                1759 non-null   category
 11  dsas_avg_diag_bilg_isamt_s   1759 non-null   float64 
 12  dsas_acd_rst_dcd             1759 non-null   int64   
 13  bas

In [ ]:
# 날짜형 변수 변환
acd1['base_ym']=pd.to_datetime(acd1['base_ym'])
acd1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1759 entries, 0 to 1758
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   nur_hosp_yn                  1759 non-null   category      
 1   ac_ctr_diff                  1759 non-null   category      
 2   hsp_avg_optt_bilg_isamt_s    1759 non-null   float64       
 3   hsp_avg_surop_bilg_isamt_s   1759 non-null   float64       
 4   ar_rclss_cd                  1759 non-null   category      
 5   fds_cust_yn                  1759 non-null   category      
 6   hspz_dys_s                   1759 non-null   float64       
 7   inamt_nvcd                   1759 non-null   int64         
 8   hsp_avg_diag_bilg_isamt_s    1759 non-null   float64       
 9   blrs_cd                      1759 non-null   int64         
 10  dsas_ltwt_gcd                1759 non-null   category      
 11  dsas_avg_diag_bilg_isamt_s   1759 non-null 

In [ ]:
acd1.drop(['dsas_acd_rst_dcd','prm_nvcd','inamt_nvcd','kcd_gcd','blrs_cd'],axis=1,
          inplace=True)
acd1.info()

#### 변수중요도 낮은애들 제거

In [ ]:
# 범주형 0,1 범주 불균형한 변수들 제거
acd1.drop(['fds_cust_yn','smrtg_5y_passed_yn','mtad_cntr_yn','heltp_pf_ntyn','urlb_fc_yn'],axis=1,inplace=True)
acd1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1759 entries, 0 to 1758
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   nur_hosp_yn                  1759 non-null   category      
 1   ac_ctr_diff                  1759 non-null   category      
 2   hsp_avg_optt_bilg_isamt_s    1759 non-null   float64       
 3   hsp_avg_surop_bilg_isamt_s   1759 non-null   float64       
 4   ar_rclss_cd                  1759 non-null   category      
 5   hspz_dys_s                   1759 non-null   float64       
 6   hsp_avg_diag_bilg_isamt_s    1759 non-null   float64       
 7   dsas_ltwt_gcd                1759 non-null   category      
 8   dsas_avg_diag_bilg_isamt_s   1759 non-null   float64       
 9   base_ym                      1759 non-null   datetime64[ns]
 10  hsp_avg_hspz_bilg_isamt_s    1759 non-null   float64       
 11  optt_blcnt_s                 1759 non-null 

In [ ]:
num_f =['bilg_isamt_s','hspz_dys_s','hsp_avg_hspz_bilg_isamt_s',
          'hsp_avg_optt_bilg_isamt_s','hsp_avg_surop_bilg_isamt_s','hsp_avg_diag_bilg_isamt_s',
          'dsas_avg_optt_bilg_isamt_s','dsas_avg_surop_bilg_isamt_s','dsas_avg_hspz_bilg_isamt_s',
          'hspz_blcnt_s','surop_blcnt_s']

acd1[num_f]

,bilg_isamt_s,hspz_dys_s,hsp_avg_hspz_bilg_isamt_s,hsp_avg_optt_bilg_isamt_s,hsp_avg_surop_bilg_isamt_s,hsp_avg_diag_bilg_isamt_s,dsas_avg_optt_bilg_isamt_s,hspz_blcnt_s,surop_blcnt_s
0,0.960461,0.000000,0.804554,1.323408,1.220151,0.723870,0.118760,0.000000,0.000000
1,1.694863,0.847284,0.776431,1.285341,1.009234,0.797192,0.118760,0.510846,0.693147
2,1.402291,1.386294,0.922352,1.008432,0.931573,0.596691,0.118760,0.510846,0.000000
3,0.780608,0.000000,0.601635,0.239253,1.204713,0.602949,0.118760,0.000000,1.098612
4,1.327048,0.523248,0.772513,0.000000,1.147212,0.275280,0.118760,0.916291,0.810930
...,...,...,...,...,...,...,...,...,...
1754,0.536786,1.620160,0.705817,0.108316,0.759824,0.453620,1.174801,0.847284,0.693147
1755,1.389614,0.000000,0.516529,0.101021,0.520934,0.085902,0.112614,0.000000,0.693147
1756,0.000000,0.372322,0.657313,0.783627,0.911198,0.613075,0.118760,0.697089,0.745977
1757,1.149939,0.075293,0.155635,0.805091,0.684359,0.453239,0.928377,1.210094,1.386294


#### catboost modeling

In [ ]:
# set x and y for CatBoost
X = acd1.drop(['target'],axis=1)
y = acd1['target']

In [ ]:
# scaling - z score


scaler = StandardScaler()
X[num_f] = scaler.fit_transform(X[num_f])

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1759 entries, 0 to 1758
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   nur_hosp_yn                  1759 non-null   category      
 1   ac_ctr_diff                  1759 non-null   category      
 2   hsp_avg_optt_bilg_isamt_s    1759 non-null   float64       
 3   hsp_avg_surop_bilg_isamt_s   1759 non-null   float64       
 4   ar_rclss_cd                  1759 non-null   category      
 5   hspz_dys_s                   1759 non-null   float64       
 6   hsp_avg_diag_bilg_isamt_s    1759 non-null   float64       
 7   base_ym                      1759 non-null   datetime64[ns]
 8   hsp_avg_hspz_bilg_isamt_s    1759 non-null   float64       
 9   optt_blcnt_s                 1759 non-null   float64       
 10  surop_blcnt_s                1759 non-null   float64       
 11  mdct_inu_rclss_dcd           1759 non-null 

In [ ]:
# split dataset
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size = 0.2, random_state=1234)
X_train.info()

In [ ]:
#import time
# train CatBoost
start = time.time()
cat_features = np.where(X.dtypes.astype("str").isin(["category","object"]))[0]
CatBoost = CatBoostClassifier(iterations=994,learning_rate=0.01,grow_policy = 'Depthwise',depth=8,
                           loss_function='MultiClass',eval_metric='TotalF1',random_seed=1234)
#model.fit(train_data,
 #         train_labels,
 #         eval_set=eval_dataset,
 #         verbose=False)

CatBoost.fit(X = X_train, y = y_train, cat_features = cat_features, eval_set=(X_valid, y_valid))

CatBoost_Runtime = time.time() - start
#print(CatBoost.get_best_iteration())

0:	learn: 0.8752258	test: 0.8537284	best: 0.8537284 (0)	total: 163ms	remaining: 2m 41s
1:	learn: 0.8987933	test: 0.8986168	best: 0.8986168 (1)	total: 250ms	remaining: 2m 4s
2:	learn: 0.9170806	test: 0.9102035	best: 0.9102035 (2)	total: 340ms	remaining: 1m 52s
3:	learn: 0.9284783	test: 0.8963824	best: 0.9102035 (2)	total: 448ms	remaining: 1m 50s
4:	learn: 0.9343778	test: 0.9046598	best: 0.9102035 (2)	total: 558ms	remaining: 1m 50s
5:	learn: 0.9301343	test: 0.8968680	best: 0.9102035 (2)	total: 644ms	remaining: 1m 45s
6:	learn: 0.9366724	test: 0.9057734	best: 0.9102035 (2)	total: 739ms	remaining: 1m 44s
7:	learn: 0.9439334	test: 0.9025975	best: 0.9102035 (2)	total: 829ms	remaining: 1m 42s
8:	learn: 0.9460744	test: 0.9116690	best: 0.9116690 (8)	total: 926ms	remaining: 1m 41s
9:	learn: 0.9483047	test: 0.9112913	best: 0.9116690 (8)	total: 1.01s	remaining: 1m 39s
10:	learn: 0.9476305	test: 0.9197862	best: 0.9197862 (10)	total: 1.12s	remaining: 1m 40s
11:	learn: 0.9490370	test: 0.9286162	best:

#### real test data불러오기

In [ ]:
testData=pd.read_csv('/content/drive/My Drive/미래에셋/test_cat2.csv')
testData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22072 entries, 0 to 22071
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   22072 non-null  int64  
 1   ID                           22072 non-null  int64  
 2   nur_hosp_yn                  22072 non-null  int64  
 3   ac_ctr_diff                  22072 non-null  int64  
 4   hsp_avg_optt_bilg_isamt_s    22072 non-null  float64
 5   hsp_avg_surop_bilg_isamt_s   22072 non-null  float64
 6   ar_rclss_cd                  22072 non-null  int64  
 7   fds_cust_yn                  22072 non-null  int64  
 8   hspz_dys_s                   22072 non-null  float64
 9   inamt_nvcd                   22072 non-null  int64  
 10  hsp_avg_diag_bilg_isamt_s    22072 non-null  float64
 11  blrs_cd                      22072 non-null  int64  
 12  dsas_ltwt_gcd                22072 non-null  int64  
 13  dsas_avg_diag_bi

In [ ]:
# 질병구분코드가 1인 행만 가져오기 # 질병구분코드 2,3,4,5로 수정
testData = testData[testData['dsas_acd_rst_dcd2']==1]
testData

,Unnamed: 0,ID,nur_hosp_yn,ac_ctr_diff,hsp_avg_optt_bilg_isamt_s,hsp_avg_surop_bilg_isamt_s,ar_rclss_cd,fds_cust_yn,hspz_dys_s,inamt_nvcd,hsp_avg_diag_bilg_isamt_s,blrs_cd,dsas_ltwt_gcd,dsas_avg_diag_bilg_isamt_s,dsas_acd_rst_dcd,base_ym,kcd_gcd,hsp_avg_hspz_bilg_isamt_s,optt_blcnt_s,mtad_cntr_yn,heltp_pf_ntyn,prm_nvcd,surop_blcnt_s,mdct_inu_rclss_dcd,dsas_avg_optt_bilg_isamt_s,isrd_age_dcd,hspz_blcnt_s,dsas_avg_surop_bilg_isamt_s,urlb_fc_yn,dsas_avg_hspz_bilg_isamt_s,smrtg_5y_passed_yn,ac_rst_diff,bilg_isamt_s,optt_nbtm_s,dsas_acd_rst_dcd2,prm_nvcd2,inamt_nvcd2,kcd_gcd_2,blrs_cd_2
32,33,32,0,6,0.668393,0.859932,1,0,0.315832,2,0.690945,11,1,0.352416,3,201912,4,0.687833,0.0,0,0,1,0.693147,3,0.921358,3,0.510846,0.272771,1,0.435542,0,0,0.759029,0.0,1,1,2,1,1
119,120,119,1,6,0.000000,0.000000,2,0,1.011601,2,0.000000,8,1,0.352416,3,201912,4,0.918728,0.0,1,0,1,0.000000,2,0.921358,9,0.510846,0.272771,0,0.435542,0,0,0.000000,0.0,1,1,2,1,5
359,360,359,1,6,0.000000,0.000000,5,0,0.606118,2,0.000000,8,1,0.352416,3,201912,4,1.745576,0.0,0,0,1,0.000000,2,0.921358,4,0.747209,0.272771,0,0.435542,0,0,0.357744,0.0,1,1,2,1,5
1108,1109,1108,0,6,0.329951,0.787639,4,0,0.463294,6,0.718766,11,1,0.352416,3,201912,4,0.856371,0.0,0,0,2,1.110442,2,0.921358,4,0.867479,0.272771,0,0.435542,0,0,1.414177,0.0,1,2,5,1,1
1157,1158,1157,0,0,0.136103,1.084716,9,0,0.000000,99,0.204898,3,1,0.352416,3,201912,4,0.528273,0.0,0,0,99,0.750378,2,0.921358,7,0.000000,0.272771,0,0.435542,0,0,0.000000,0.0,1,99,99,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20818,20819,20818,0,6,0.009554,0.812218,9,0,0.164242,4,0.647313,11,1,0.352416,3,201912,4,0.429572,0.0,0,0,1,0.693147,2,0.921358,3,0.510846,0.272771,0,0.435542,0,0,0.573913,0.0,1,1,4,1,1
21167,21168,21167,0,0,0.489132,1.149274,9,0,0.000000,99,0.415679,3,1,0.352416,3,201912,4,0.772651,0.0,0,0,99,0.920123,2,0.921358,7,0.000000,0.272771,0,0.435542,0,0,0.000000,0.0,1,99,99,1,4
21448,21449,21448,0,0,0.547543,1.161837,9,0,0.000000,99,0.453366,3,1,0.352416,3,201912,4,0.815542,0.0,0,0,99,0.951194,2,0.921358,7,0.000000,0.272771,0,0.435542,0,0,0.000000,0.0,1,99,99,1,4
21839,21840,21839,0,6,1.260164,1.319299,5,0,0.000000,3,0.842472,3,1,0.352416,3,201912,4,0.728514,0.0,0,0,1,0.693147,3,0.921358,2,0.000000,0.272771,0,0.435542,0,0,1.150382,0.0,1,1,3,1,4


In [ ]:
# factor형 변수 변환
cat_f = ['dsas_ltwt_gcd','ar_rclss_cd','mdct_inu_rclss_dcd','nur_hosp_yn', 'isrd_age_dcd',
          'fds_cust_yn','smrtg_5y_passed_yn','mtad_cntr_yn','heltp_pf_ntyn',
          'ac_ctr_diff','ac_rst_diff','urlb_fc_yn','kcd_gcd_2','blrs_cd_2',
          'dsas_acd_rst_dcd2','prm_nvcd2','inamt_nvcd2']

for colnm in cat_f:
  testData[colnm]= testData[colnm].astype('category')

testData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116 entries, 32 to 21889
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   Unnamed: 0                   116 non-null    int64   
 1   ID                           116 non-null    int64   
 2   nur_hosp_yn                  116 non-null    category
 3   ac_ctr_diff                  116 non-null    category
 4   hsp_avg_optt_bilg_isamt_s    116 non-null    float64 
 5   hsp_avg_surop_bilg_isamt_s   116 non-null    float64 
 6   ar_rclss_cd                  116 non-null    category
 7   fds_cust_yn                  116 non-null    category
 8   hspz_dys_s                   116 non-null    float64 
 9   inamt_nvcd                   116 non-null    int64   
 10  hsp_avg_diag_bilg_isamt_s    116 non-null    float64 
 11  blrs_cd                      116 non-null    int64   
 12  dsas_ltwt_gcd                116 non-null    category
 13  ds

In [ ]:
testData.drop(['Unnamed: 0','ID','inamt_nvcd','blrs_cd','dsas_acd_rst_dcd','kcd_gcd','prm_nvcd'],
              axis=1, inplace=True)
testData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116 entries, 32 to 21889
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   nur_hosp_yn                  116 non-null    category
 1   ac_ctr_diff                  116 non-null    category
 2   hsp_avg_optt_bilg_isamt_s    116 non-null    float64 
 3   hsp_avg_surop_bilg_isamt_s   116 non-null    float64 
 4   ar_rclss_cd                  116 non-null    category
 5   fds_cust_yn                  116 non-null    category
 6   hspz_dys_s                   116 non-null    float64 
 7   hsp_avg_diag_bilg_isamt_s    116 non-null    float64 
 8   dsas_ltwt_gcd                116 non-null    category
 9   dsas_avg_diag_bilg_isamt_s   116 non-null    float64 
 10  base_ym                      116 non-null    int64   
 11  hsp_avg_hspz_bilg_isamt_s    116 non-null    float64 
 12  optt_blcnt_s                 116 non-null    float64 
 13  mt

In [ ]:

X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1407 entries, 929 to 815
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   nur_hosp_yn                  1407 non-null   category      
 1   ac_ctr_diff                  1407 non-null   category      
 2   hsp_avg_optt_bilg_isamt_s    1407 non-null   float64       
 3   hsp_avg_surop_bilg_isamt_s   1407 non-null   float64       
 4   ar_rclss_cd                  1407 non-null   category      
 5   hspz_dys_s                   1407 non-null   float64       
 6   hsp_avg_diag_bilg_isamt_s    1407 non-null   float64       
 7   base_ym                      1407 non-null   datetime64[ns]
 8   hsp_avg_hspz_bilg_isamt_s    1407 non-null   float64       
 9   optt_blcnt_s                 1407 non-null   float64       
 10  surop_blcnt_s                1407 non-null   float64       
 11  mdct_inu_rclss_dcd           1407 non-null

In [ ]:
# 범주형 0,1 범주 불균형한 변수들 제거
testData.drop(['fds_cust_yn','smrtg_5y_passed_yn','mtad_cntr_yn','heltp_pf_ntyn','urlb_fc_yn'],axis=1,inplace=True)
testData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116 entries, 32 to 21889
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   nur_hosp_yn                  116 non-null    category
 1   ac_ctr_diff                  116 non-null    category
 2   hsp_avg_optt_bilg_isamt_s    116 non-null    float64 
 3   hsp_avg_surop_bilg_isamt_s   116 non-null    float64 
 4   ar_rclss_cd                  116 non-null    category
 5   hspz_dys_s                   116 non-null    float64 
 6   hsp_avg_diag_bilg_isamt_s    116 non-null    float64 
 7   dsas_ltwt_gcd                116 non-null    category
 8   dsas_avg_diag_bilg_isamt_s   116 non-null    float64 
 9   base_ym                      116 non-null    int64   
 10  hsp_avg_hspz_bilg_isamt_s    116 non-null    float64 
 11  optt_blcnt_s                 116 non-null    float64 
 12  surop_blcnt_s                116 non-null    float64 
 13  md

In [ ]:
testData['base_ym']=pd.to_datetime(testData['base_ym'])
testData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116 entries, 32 to 21889
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   nur_hosp_yn                  116 non-null    category      
 1   ac_ctr_diff                  116 non-null    category      
 2   hsp_avg_optt_bilg_isamt_s    116 non-null    float64       
 3   hsp_avg_surop_bilg_isamt_s   116 non-null    float64       
 4   ar_rclss_cd                  116 non-null    category      
 5   hspz_dys_s                   116 non-null    float64       
 6   hsp_avg_diag_bilg_isamt_s    116 non-null    float64       
 7   base_ym                      116 non-null    datetime64[ns]
 8   hsp_avg_hspz_bilg_isamt_s    116 non-null    float64       
 9   optt_blcnt_s                 116 non-null    float64       
 10  surop_blcnt_s                116 non-null    float64       
 11  mdct_inu_rclss_dcd           116 non-null 

In [ ]:
print(X_train.columns)
print(testData.columns)

Index(['nur_hosp_yn', 'ac_ctr_diff', 'hsp_avg_optt_bilg_isamt_s',
       'hsp_avg_surop_bilg_isamt_s', 'ar_rclss_cd', 'hspz_dys_s',
       'hsp_avg_diag_bilg_isamt_s', 'base_ym', 'hsp_avg_hspz_bilg_isamt_s',
       'optt_blcnt_s', 'surop_blcnt_s', 'mdct_inu_rclss_dcd',
       'dsas_avg_optt_bilg_isamt_s', 'isrd_age_dcd', 'hspz_blcnt_s',
       'dsas_avg_surop_bilg_isamt_s', 'ac_rst_diff', 'bilg_isamt_s',
       'optt_nbtm_s', 'prm_nvcd2', 'inamt_nvcd2', 'kcd_gcd_2', 'blrs_cd_2'],
      dtype='object')
Index(['nur_hosp_yn', 'ac_ctr_diff', 'hsp_avg_optt_bilg_isamt_s',
       'hsp_avg_surop_bilg_isamt_s', 'ar_rclss_cd', 'hspz_dys_s',
       'hsp_avg_diag_bilg_isamt_s', 'base_ym', 'hsp_avg_hspz_bilg_isamt_s',
       'optt_blcnt_s', 'surop_blcnt_s', 'mdct_inu_rclss_dcd',
       'dsas_avg_optt_bilg_isamt_s', 'isrd_age_dcd', 'hspz_blcnt_s',
       'dsas_avg_surop_bilg_isamt_s', 'ac_rst_diff', 'bilg_isamt_s',
       'optt_nbtm_s', 'prm_nvcd2', 'inamt_nvcd2', 'kcd_gcd_2', 'blrs_cd_2'],
      dt

In [ ]:
testData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116 entries, 32 to 21889
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   nur_hosp_yn                  116 non-null    category      
 1   ac_ctr_diff                  116 non-null    category      
 2   hsp_avg_optt_bilg_isamt_s    116 non-null    float64       
 3   hsp_avg_surop_bilg_isamt_s   116 non-null    float64       
 4   ar_rclss_cd                  116 non-null    category      
 5   hspz_dys_s                   116 non-null    float64       
 6   hsp_avg_diag_bilg_isamt_s    116 non-null    float64       
 7   base_ym                      116 non-null    datetime64[ns]
 8   hsp_avg_hspz_bilg_isamt_s    116 non-null    float64       
 9   optt_blcnt_s                 116 non-null    float64       
 10  surop_blcnt_s                116 non-null    float64       
 11  mdct_inu_rclss_dcd           116 non-null 

In [ ]:
# scaling
testData[num_f] = scaler.transform(testData[num_f])

In [ ]:
testData[num_f]

,bilg_isamt_s,hspz_dys_s,hsp_avg_hspz_bilg_isamt_s,hsp_avg_optt_bilg_isamt_s,hsp_avg_surop_bilg_isamt_s,hsp_avg_diag_bilg_isamt_s,dsas_avg_optt_bilg_isamt_s,hspz_blcnt_s,surop_blcnt_s
32,-0.092273,-0.154208,-0.110511,0.144680,0.343055,0.640369,1.063956,0.146960,0.532751
119,-0.909341,0.793135,0.269177,-1.003037,-2.091093,-1.887936,1.063956,0.146960,-1.040592
359,-0.524242,0.241038,1.628862,-1.003037,-2.091093,-1.887936,1.063956,0.707681,-1.040592
1108,0.612972,0.046572,0.166636,-0.436469,0.138420,0.742173,1.063956,0.992995,1.479948
1157,-0.909341,-0.584238,-0.372895,-0.769330,0.979335,-1.138173,1.063956,-1.064911,0.662656
...,...,...,...,...,...,...,...,...,...
20818,-0.291543,-0.360609,-0.535200,-0.986631,0.207995,0.480710,1.063956,0.146960,0.532751
21167,-0.909341,-0.584238,0.028966,-0.163135,1.162074,-0.366882,1.063956,-1.064911,1.047953
21448,-0.909341,-0.584238,0.099496,-0.062835,1.197637,-0.228980,1.063956,-1.064911,1.118480
21839,0.329006,-0.584238,-0.043614,1.160825,1.643354,1.194837,1.063956,-1.064911,0.532751


In [ ]:
test_pred=CatBoost.predict(testData)
test_pred

array([[1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [1],
       [2],
       [1],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [1],
       [2],
       [1],
       [1],
       [2],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [1],
       [2],
       [2],
       [2],
       [2],
       [1],
       [2],
       [2],
       [1],
       [2],
       [2],
       [2],
       [2],
       [1],
       [2],
       [2],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [1],
       [2],
       [2],
       [2],
       [1],
       [2],
       [2],
       [1],
       [2],
       [2],
       [2],
       [1],
       [2],
       [2],
       [1],
       [1],
       [2],
       [1],
       [2],
       [2],
    

In [ ]:
test_pred2=np.zeros(len(test_pred))
test_pred2

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
for i in range(len(test_pred)):
  test_pred2[i]= list(test_pred[i])[0]

In [ ]:
test_pred2

array([1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 1., 2., 1., 1., 2., 1., 2.,
       1., 2., 2., 2., 2., 2., 2., 1., 2., 1., 1., 2., 1., 1., 1., 2., 2.,
       2., 1., 2., 2., 2., 2., 1., 2., 2., 1., 2., 2., 2., 2., 1., 2., 2.,
       1., 1., 1., 2., 2., 2., 2., 1., 2., 1., 2., 1., 1., 2., 2., 2., 1.,
       2., 2., 1., 2., 2., 2., 1., 2., 2., 1., 1., 2., 1., 2., 2., 1., 1.,
       2., 1., 1., 2., 2., 1., 1., 1., 2., 1., 2., 2., 1., 1., 2., 1., 1.,
       2., 2., 2., 2., 2., 2., 1., 2., 1., 1., 2., 2., 1., 2.])

In [ ]:
cat_res=pd.DataFrame({'target':test_pred2})
cat_res['target']=cat_res['target'].astype('int')
cat_res['target']=cat_res['target'].astype('category')
cat_res

,target
0,1.0
1,1.0
2,1.0
3,2.0
4,2.0
...,...
111,1.0
112,2.0
113,2.0
114,1.0


In [ ]:
cat_res.to_csv('catboost_cat2_parameter수정_biased,상관관계변수제거_acd1.csv')